In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from debug_utils import get_omodel, get_peft_ckpt, generate_outputs, adjust_input_apf, adjust_input_stack
from datasets import load_dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Start by checking stuff out for the webgpt model

In [ ]:
with open('../basepath.txt', 'r') as file:
    bpath = file.read().strip()

with open('../apfpath.txt', 'r') as file:
    apfpath = file.read().strip()

In [ ]:
# for webgpt 
basename = apfpath+"apfarm_models/sft10k/"
origmodel, toker = get_omodel(basename)

In [ ]:
# for stack stuff
basename = bpath+"stack-llama/models/sft/"
origmodel, toker = get_omodel(basename)

In [ ]:
generation_kwargs = {
    "min_new_tokens": -1,
    "max_new_tokens":512,
    #"top_k": 0.0,
    "top_p": 1,
    #"temperature": 0.9,
    "do_sample": True,
    #"repetition_penalty": 1.2,
    #"pad_token_id": tokenizer.pad_token_id,
    #"eos_token_id": tokenizer.eos_token_id,
}

In [ ]:
# for webgpt
inps = load_dataset("openai/webgpt_comparisons", split="test")

In [ ]:
inps = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")

In [ ]:
ind = 20
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
print(len(toks[0]))
print(outs[0])

In [ ]:
#SFT Starts out reasonably short

In [ ]:
adaptpath = bpath+"/webgpt-llama/checkpoints/wgptapsft/step_125"
amod = get_peft_ckpt(origmodel, adaptpath)

In [ ]:
ind = 40
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, amod, generation_kwargs)
print(len(toks[0]))
print(outs[0])

In [ ]:
# Testing stuff for Stack Model (OOF the KL was too high to be a reproduction...)

In [ ]:
lens = []
for ind in range(0, 20):
    qtemp = adjust_input_stack(inps[ind]['question'])
    toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
    print(len(toks[0]))
    print(outs[0])
    lens.append(len(toker(outs[0].split("Answer:")[1]).input_ids))

In [ ]:
adaptpath = bpath+"/stack-llama/checkpoints/lenhacksanity/step_125"
amod = get_peft_ckpt(origmodel, adaptpath)

In [ ]:
lens = []
for ind in range(0, 20):
    qtemp = adjust_input_stack(inps[ind]['question'])
    toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
    print(len(toks[0]))
    print(outs[0])
    lens.append(len(toker(outs[0].split("Answer:")[1]).input_ids))

In [ ]:
sum(lens)/len(lens)

In [ ]:
lens